# Primeros pasos con AMPL en Python mediante la interfaz *amplpy*

Podéis encontrar multitud de información sobre cómo trabajar con AMPL en el entorno de trabajo colaborativo Google colab en https://colab.ampl.com/.

Instalamos en primer lugar la API *amplpy*:

In [1]:
%pip install -q amplpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.9 MB/s eta 0:00:00


Y ahora instanciamos un objeto "AMPL" y descargamos el "solver" necesario para resolver nuestro problema de optimización.

In [2]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


Ahora ya tenemos instalado todo lo necesario para resolver un problema de Programación Lineal como, por ejemplo, el **problema de la dieta** que vimos el primer día de clase.

# Problema de la dieta

Como ya sabemos, el problema de la dieta consiste en determinar las cantidades de distintos nutrientes que deben ingerirse para asegurar ciertas condiciones de nutrición y minimizar el coste de compra de los nutrientes. Consideremos el caso que vimos en clase: Tenemos los cuatro tipos de nutrientes y los cinco alimentos que se presentan en la siguiente tabla, donde se recogen los contenidos nutritivos de cada alimento (por cada 100 gramos), junto con las cantidades mínimas requeridas para cada tipo de nutriente.

|  Nutriente    | Cantidad <br />  requerida   | Maíz A   | Avena  | Maíz B  | Salvado | Linaza|
| ------------  | --------     | ---      | ---    | ---     | ----    | ---   |
| DN            | 74.2         | 78.6     | 70.1   | 80.1    | 67.2    | 77.0  |
| DP            | 14.7         | 6.50     | 9.40   | 8.80    | 13.7    | 30.4  |
| Ca            | 0.14         | 0.02     | 0.09   | 0.03    | 0.14    | 0.41  |
| Ph            | 0.55         | 0.27     | 0.34   | 0.30    | 1.29    | 0.86  |

El coste de cada alimento (por cada 100g) se indica en esta otra tabla, donde también les asignamos un acrónimo.


|Acrónimo |    Alimento        | Coste € |
|---------| -----------------  | ------- |
|mA       |  Maíz A            | 1       |
|A        |  Avena             | 0.5     |
|mB       |  Maíz B            | 2       |
|S        |  Salvado           | 1.2     |
|L        |  Linaza            | 3       |


Ahora, llamaremos $x_{mA}$, $x_{A}$, $x_{mB}$, $x_{S}$ y $x_{L}$ a las cantidades de maíz A, avena, maíz B, salvado y linaza que deben formar parte de la dieta, de forma que el coste total de dicha dieta vendrá dado por
$$coste\_total = x_{mA} + 0.5x_{A} + 2 x_{mB} + 1.2 x_{S} + 3 x_{L}$$

Asimismo, para cumplir con la cantidad requerida de nutriente DN en la dieta, deberemos imponer en nuestro modelo la siguiente restricción:

$$78.6x_{mA} + 70.1x_{A}+80.1x_{mB}+67.2x_{S} + 77x_{L} \geq 74.2$$

Las cantidades mínimas para los otros tres nutrientes darán lugar a restricciones análogas.

Por supuesto, las cantidades de alimento que se incorporarán en la dieta habrán de ser no negativas, esto es:

$$x_{mA}, x_{A}, x_{mB}, x_{S}, x_{L} \geq 0$$

Ya estamos listos para construir nuestro modelo y resolver el problema con la ayuda de AMPL.


### Construcción del modelo en el objeto "ampl":

In [3]:
ampl.eval(r"""
var x_mA >= 0;
var x_A >= 0;
var x_mB >= 0;
var x_S >= 0;
var x_L >= 0;

minimize coste_total:
    x_mA + 0.5*x_A + 2*x_mB + 1.2*x_S + 3*x_L;
s.t. cant_minima_DN:
    78.6*x_mA + 70.1*x_A + 80.1*x_mB + 67.2*x_S + 77*x_L >= 74.2;
     cant_minima_DP:
    6.50*x_mA + 9.40*x_A + 8.80*x_mB + 13.7*x_S + 30.4*x_L >= 14.7;
     cant_minima_Ca:
    0.02*x_mA + 0.09*x_A + 0.03*x_mB + 0.14*x_S + 0.41*x_L >= 0.14;
     cant_minima_Ph:
    0.27*x_mA + 0.34*x_A + 0.30*x_mB + 1.29*x_S + 0.86*x_L >= 0.55;
    """)

### Resolución del modelo:

In [4]:
ampl.solve(solver="highs") # Resolvemos con el solver "highs"
assert ampl.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

HiGHS 1.7.1: HiGHS 1.7.1: optimal solution; objective 0.7927691484
3 simplex iterations
0 barrier iterations


Extraemos del objeto "ampl" los valores óptimos de las variables de decisión y su coste:

In [5]:
x_mA = ampl.get_variable("x_mA").value()
x_A = ampl.get_variable("x_A").value()
x_mB = ampl.get_variable("x_mB").value()
x_S = ampl.get_variable("x_S").value()
x_L = ampl.get_variable("x_L").value()

print(f"x_mA: {x_mA:.3f}")
print(f"x_A: {x_A:.3f}")
print(f"x_mB: {x_mB:.3f}")
print(f"x_S: {x_S:.3f}")
print(f"x_L: {x_L:.3f}")

# Ídem para el valor objetivo de la solución óptima:
coste_total = ampl.get_objective("coste_total").value()
print(f"Coste total: {coste_total:.3f}")

x_mA: 0.000
x_A: 1.530
x_mB: 0.000
x_S: 0.023
x_L: 0.000
Coste total: 0.793


También podemos obtener el valor (del lado izquierdo) de una restricción evaluada en la solución óptima:

In [6]:
cant_minima_DN =ampl.get_constraint("cant_minima_DN").body()
print(f"cant_minima_DN: {cant_minima_DN:.3f}")

cant_minima_DN: 108.819


¡E incluso su variable dual (marginal, multiplicador de Lagrange o precio sombra)!

In [7]:
cant_minima_DN_dual =ampl.get_constraint("cant_minima_DN").dual()
print(f"Variable dual de la restricción cant_minima_DN: {cant_minima_DN_dual:.3f}")

Variable dual de la restricción cant_minima_DN: 0.000


In [8]:
ampl.solve(solver="cplex", verbose = True, cplex_options = "alg:method 0 pre:solve 0") # Resolvemos con el solver "cplex"
assert ampl.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1:   alg:method = 0
  pre:solve = 0
 - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_Preprocessing_Presolve                  0
 - CPXPARAM_LPMethod                                1
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 0.7927691484
0 simplex iterations


In [9]:
x_mA = ampl.get_variable("x_mA").value()
x_A = ampl.get_variable("x_A").value()
x_mB = ampl.get_variable("x_mB").value()
x_S = ampl.get_variable("x_S").value()
x_L = ampl.get_variable("x_L").value()
coste_total = ampl.get_objective("coste_total").value()
print(f"x_mA: {x_mA:.3f}")
print(f"x_A: {x_A:.3f}")
print(f"x_mB: {x_mB:.3f}")
print(f"x_S: {x_S:.3f}")
print(f"x_L: {x_L:.3f}")
print(f"Coste total: {coste_total:.3f}")

x_mA: 0.000
x_A: 1.530
x_mB: 0.000
x_S: 0.023
x_L: 0.000
Coste total: 0.793


## Construcción de un modelo general


Ahora vamos a usar las capacidades de AMPL para generalizar el modelo LP anterior para el caso en que tenemos un número $n_a$ de alimentos y un número $n_n$ de nutrientes. Para ello, utilizaremos conjuntos de índices, así como paramétros en que habremos de introducir los datos del problema.

In [10]:
ampl.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set ALIM;                       # Conjunto de alimentos
set NUTR;                       # Conjunto de nutrientes
param n_min {NUTR} >= 0;        # Cantidad mínima de nutrientes en la dieta
param coste {ALIM} >= 0;        # Coste de cada alimento (por cada 100 g)
param cont {ALIM, NUTR} >= 0;   # Contenido de nutriente en cada alimento

var x {ALIM} >= 0;              # Cantidad de cada alimento presente la dieta
                                # (múltiplo de 100 g)

minimize coste_total:
    sum{a in ALIM} coste[a]*x[a];
s.t. cant_minima {n in NUTR}:
    sum{a in ALIM} cont[a,n]*x[a] >= n_min[n];
""")

Pasamos a introducir los valores numéricos específicos que toman los parámetros del modelo para el ejemplo concreto que queremos resolver.

In [11]:
ampl.eval(r"""
data;

set ALIM := mA A mB S L;
set NUTR := DN DP Ca Ph;

param n_min := DN 74.2 DP 14.7 Ca 0.14 Ph 0.55;
param coste :=
    mA 1
    A 0.5
    mB 2
    L 3
    S 1.2
    ;

param cont (tr):
        mA    A     mB    S     L :=
  DN    78.6  70.1  80.1  67.2  77.0
  DP    6.50  9.40  8.80  13.7  30.4
  Ca    0.02  0.09  0.03  0.14  0.41
  Ph    0.27  0.34  0.30  1.29  0.86
;

""")

Y pasamos a resolver el modelo.

In [12]:
ampl.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert ampl.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1:   alg:method = 0
  pre:solve = 0
 - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_Preprocessing_Presolve                  0
 - CPXPARAM_LPMethod                                1
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 0.7927691484
4 simplex iterations


Para extaer información sobre el problema resuelto, en concreto, los valores que adoptan las variables de decisión, hemos de tener en cuenta que dichas variables aparacen ahora en el modelo indexadas por el conjunto ALIM (es decir, la variable "x" no es un simple valor númerico real). Por ello, para extraer su valor recurrimos a su transformación a "data frame" por medio de *amplpy*.

In [13]:
x = ampl.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
coste_total = ampl.get_objective("coste_total").value()
df_x = df_x.round(3)
print(df_x)
print(f"Coste total: {coste_total:.3f}")

    x.val
A   1.530
L   0.000
S   0.023
mA  0.000
mB  0.000
Coste total: 0.793


La forma anterior en que hemos introducido los datos de entrada del problema de la dieta (esto es, los valores de los parámetros para la instancia específica de dicho problema que queremos resolver) corresponde al formato y sintásis que el entorno AMPL utiliza para tal propósito. Sin embargo, con la API *amplpy* podemos introducir los datos de entrada y asignar valores a parámetros utilizando listas y diccionarios de Python, así como data frames de Pandas. Veámoslo.

In [14]:
def preparar_datos():
    import pandas as pd
    import numpy as np

    # Creamos un data frame que contenga el coste de los alimentos
    alimentos_df = pd.DataFrame(
        [
            ("MaizA", 1),
            ("Avena", 0.5),
            ("MaizB", 2),
            ("Salvado", 1.2),
            ("Linaza", 3),
        ],
        columns=["ALIMENTO", "coste"],
    ).set_index("ALIMENTO")

    # Creamos un data frame para la cantidad mínima de nutrientes que se
    # requiere en la dieta
    nutr_df = pd.DataFrame(
        [
            ("DN", 74.2),
            ("DP", 14.7),
            ("Ca", 0.14),
            ("Ph", 0.55),
        ],
        columns=["NUTR", "n_min"],
    ).set_index("NUTR")

    # Creamos un data frame para la cantidad de nutrientes que cada
    # alimento contiene
    cont_df = pd.DataFrame(
        np.array(
            [
                [78.6, 70.1, 80.1, 67.2, 77.0],
                [6.50, 9.40, 8.80, 13.7, 30.4],
                [0.02, 0.09, 0.03, 0.14, 0.41],
                [0.27, 0.34, 0.30, 1.29, 0.86],
            ]
        ),
        columns= alimentos_df.index.to_list(),
        index  = nutr_df.index.to_list(),
    )
    cont_df = cont_df.transpose() # OJO, que los ejes (índices) deben coincidir
                                  # con los especificados en la definición del
                                  # parámetro "cont" en el modelo, (ALIM, NUTR)
    return alimentos_df, nutr_df, cont_df

In [15]:
# Volvemos a cargar el modelo para asegurar que borramos el contenido del obnjeto "ampl"

ampl.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set ALIM;                       # Conjunto de alimentos
set NUTR;                       # Conjunto de nutrientes
param n_min {NUTR} >= 0;        # Cantidad mínima de nutrientes en la dieta
param coste {ALIM} >= 0;        # Coste de cada alimento (por cada 100 g)
param cont {ALIM, NUTR} >= 0;   # Contenido de nutriente en cada alimento

var x {ALIM} >= 0;              # Cantidad de cada alimento presente en la dieta
                                # (múltiplo de 100 g)

minimize coste_total:
    sum{a in ALIM} coste[a]*x[a];
s.t. cant_minima {n in NUTR}:
    sum{a in ALIM} cont[a,n]*x[a] >= n_min[n];
""")

In [16]:
# Cargamos los datos:

# Load the data from pandas.DataFrame objects:
alimentos_df, nutr_df, cont_df = preparar_datos()

# 1. Enviamos los datos de "alimentos_df" a AMPL e inicializamos el conjunto de índices "ALIM"
ampl.set_data(alimentos_df, "ALIM")
# 2. Enviamos los datos de "nutr_df" a AMPL e inicializamos el conjunto de índices "NUTR"
ampl.set_data(nutr_df, "NUTR")
# 3. Fijamos los valores del parámetro "cont" usando "cont_df"
ampl.get_parameter("cont").set_values(cont_df)

In [17]:
ampl.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert ampl.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1:   alg:method = 0
  pre:solve = 0
 - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_Preprocessing_Presolve                  0
 - CPXPARAM_LPMethod                                1
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 0.7927691484
4 simplex iterations


In [18]:
x = ampl.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
coste_total = ampl.get_objective("coste_total").value()
df_x = df_x.round(3)
print(df_x)
print(f"Coste total: {coste_total:.3f}")

         x.val
Avena    1.530
Linaza   0.000
MaizA    0.000
MaizB    0.000
Salvado  0.023
Coste total: 0.793


Y ahora utilizando listas y diccionarios:

In [19]:
# Volvemos a cargar el modelo para asegurar que borramos el contenido del obnjeto "ampl"

ampl.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set ALIM;                       # Conjunto de alimentos
set NUTR;                       # Conjunto de nutrientes
param n_min {NUTR} >= 0;        # Cantidad mínima de nutrientes en la dieta
param coste {ALIM} >= 0;        # Coste de cada alimento (por cada 100 g)
param cont {ALIM, NUTR} >= 0;   # Contenido de nutriente en cada alimento

var x {ALIM} >= 0;              # Cantidad de cada alimento presente en la dieta
                                # (múltiplo de 100 g)

minimize coste_total:
    sum{a in ALIM} coste[a]*x[a];
s.t. cant_minima {n in NUTR}:
    sum{a in ALIM} cont[a,n]*x[a] >= n_min[n];
""")

In [20]:
# Introducimos el coste de los alimentos como diccionario
# alimentos[alim] = coste
alimentos = {
    "maizA": 1,
    "avena": 0.5,
    "maizB": 2,
    "salvado": 1.2,
    "linaza": 3,
}
# Introducimos la cantidad mínima requerida de nutriente como lista
nutrientes = ["DN", "DP", "Ca", "Ph"]
n_min =[74.2, 14.7, 0.14, 0.55]

ampl.set["ALIM"] = list(alimentos.keys())
ampl.param["coste"] = {alim: coste for alim, coste in alimentos.items()}
ampl.set["NUTR"] = nutrientes
ampl.param["n_min"] = {nutrientes[i]: j for i, j in enumerate(n_min)}

# Introducimos el contenido nutritivo de cada alimento como una lista de listas
contenido = [
    [78.6, 70.1, 80.1, 67.2, 77.0],
    [6.50, 9.40, 8.80, 13.7, 30.4],
    [0.02, 0.09, 0.03, 0.14, 0.41],
    [0.27, 0.34, 0.30, 1.29, 0.86],
]
ampl.param["cont"] = {
    (alimento, nutriente): contenido[i][j]
    for i, nutriente in enumerate(nutrientes)
    for j, alimento in enumerate(alimentos)
}

In [21]:
ampl.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert ampl.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1:   alg:method = 0
  pre:solve = 0
 - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_Preprocessing_Presolve                  0
 - CPXPARAM_LPMethod                                1
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 0.7927691484
4 simplex iterations


In [22]:
x = ampl.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
coste_total = ampl.get_objective("coste_total").value()
df_x = df_x.round(3)
print(df_x)
print(f"Coste total: {coste_total:.3f}")

         x.val
avena    1.530
linaza   0.000
maizA    0.000
maizB    0.000
salvado  0.023
Coste total: 0.793


**Ejercicio**: Extiende el modelo LP con el que hemos venido trabajando para resolver el problema de la dieta de forma tal que se considere una cantidad mínima y máxima de alimento que se puede usar para elaborar la dieta (a_min y a_max, respectivamente) así como una cantidad máxima de nutriente que puede estar presente en la misma (n_max). Resuelve entonces dicho modelo para los siguientes datos de entrada:

|Acrónimo |    Alimento        | Coste € | Mínimo | Máximo|
|---------| -----------------  | ------- |--------|-------|
|mA       |  Maíz A            | 1       | 0.20   |  1.00 |
|A        |  Avena             | 0.5     | 0.30   |  1.20 |
|mB       |  Maíz B            | 2       | 0.10   |  2.00 |
|S        |  Salvado           | 1.2     | 0.50   |  3.50 |
|L        |  Linaza            | 3       | 0.00   |  0.75 |

|  Nutriente    | Cantidad <br />  requerida | Cantidad <br /> máxima  | Maíz A   | Avena  | Maíz B  | Salvado | Linaza|
| ------------  | --------     |-------   | ---      | ---    | ---     | ----    | ---   |
| DN            | 74.2         |120.1     | 78.6     | 70.1   | 80.1    | 67.2    | 77.0  |
| DP            | 14.7         |18.4     | 6.50     | 9.40   | 8.80    | 13.7    | 30.4  |
| Ca            | 0.14         |0.95      | 0.02     | 0.09   | 0.03    | 0.14    | 0.41  |
| Ph            | 0.55         |1.23      | 0.27     | 0.34   | 0.30    | 1.29    | 0.86  |

In [23]:
# Volvemos a cargar el modelo para asegurar que borramos el contenido del obnjeto "ampl"

ampl.eval(r"""
reset;                                      # Para limpiar cualquier modelo previo
set ALIM;                                   # Conjunto de alimentos
set NUTR;                                   # Conjunto de nutrientes
param n_min {NUTR} >= 0;                    # Cantidad mínima de nutrientes en la dieta
param n_max {n in NUTR} >= n_min[n];        # Cantidad máxima de nutrientes en la dieta
param a_min {ALIM} >= 0;                    # Cantidad mínima de cada alimento en la dieta
param a_max {a in ALIM} >= a_min[a];        # Cantidad máxima de cada alimento en la dieta
param coste {ALIM} >= 0;                    # Coste de cada alimento (por cada 100 g)
param cont {ALIM, NUTR} >= 0;               # Contenido de nutriente en cada alimento

var x {a in ALIM} >= a_min[a], <= a_max[a]; # Cantidad de cada alimento presente en la dieta
                                            # (múltiplo de 100 g)

minimize coste_total:
    sum{a in ALIM} coste[a]*x[a];
s.t. cant_minima {n in NUTR}:
   n_min[n]<= sum{a in ALIM} cont[a,n]*x[a] <= n_max[n];
""")

In [24]:
def preparar_datos_ext():
    import pandas as pd
    import numpy as np

    # Creamos un data frame que contenga el coste de los alimentos y sus cantidades mínima y máxima
    alimentos_df = pd.DataFrame(
        [
            ("MaizA", 1, 0.2, 1),
            ("Avena", 0.5, 0.3, 1.2),
            ("MaizB", 2, 0.1, 2),
            ("Salvado", 1.2, 0.5, 3.5),
            ("Linaza", 3, 0, 0.75),
        ],
        columns=["ALIMENTO", "coste", "a_min", "a_max"],
    ).set_index("ALIMENTO")

    # Creamos un data frame para las cantidades mínima y máxima de nutrientes que se requiere en la dieta
    nutr_df = pd.DataFrame(
        [
            ("DN", 74.2, 120.1),
            ("DP", 14.7, 18.4),
            ("Ca", 0.14, 0.95),
            ("Ph", 0.55, 1.23),
        ],
        columns=["NUTR", "n_min", "n_max"],
    ).set_index("NUTR")

    # Creamos un data frame para la cantidad de nutrientes que cada
    # alimento contiene
    cont_df = pd.DataFrame(
        np.array(
            [
                [78.6, 70.1, 80.1, 67.2, 77.0],
                [6.50, 9.40, 8.80, 13.7, 30.4],
                [0.02, 0.09, 0.03, 0.14, 0.41],
                [0.27, 0.34, 0.30, 1.29, 0.86],
            ]
        ),
        columns= alimentos_df.index.to_list(),
        index  = nutr_df.index.to_list(),
    )
    cont_df = cont_df.transpose() # OJO, que los ejes (índices) deben coincidir
                                  # con los especificados en la definición del
                                  # parámetro "cont" en el modelo, (ALIM, NUTR)
    return alimentos_df, nutr_df, cont_df

In [25]:
# Cargamos los datos:

# Load the data from pandas.DataFrame objects:
alimentos_df, nutr_df, cont_df = preparar_datos_ext()

# 1. Enviamos los datos de "alimentos_df" a AMPL e inicializamos el conjunto de índices "ALIM"
ampl.set_data(alimentos_df, "ALIM")
# 2. Enviamos los datos de "nutr_df" a AMPL e inicializamos el conjunto de índices "NUTR"
ampl.set_data(nutr_df, "NUTR")
# 3. Fijamos los valores del parámetro "cont" usando "cont_df"
ampl.get_parameter("cont").set_values(cont_df)

In [26]:
ampl.solve(solver="cplex") # Resolvemos con el solver "cplex"
print(ampl.solve_result)


CPLEX 22.1.1:   alg:method = 0
  pre:solve = 0
 - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_Preprocessing_Presolve                  0
 - CPXPARAM_LPMethod                                1
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 1.35
4 simplex iterations
solved


In [27]:
x = ampl.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = df = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
coste_total = ampl.get_objective("coste_total").value()
df_x = df_x.round(3)
print(df_x)
print(f"Coste total: {coste_total:.3f}")

         x.val
Avena      0.7
Linaza     0.0
MaizA      0.2
MaizB      0.1
Salvado    0.5
Coste total: 1.350


**Ejercicio**: Modifica de la forma más sencilla que se te ocurra (por ejemplo, tratando de minimizar el mínimo número de cambios) el valor de los parámetros de la instancia del problema anterior para que el programa LP resultante se vuelva infactible, no acotado o tenga múltiples soluciones óptimas. En este último caso, proprociona al menos dos soluciones óptimas distintas.